In [31]:
import tensorflow as tf
import mf
import sudoku
import numpy as np
np.set_printoptions(precision=2, suppress=True)

In [170]:
reload(mf)

<module 'mf' from 'mf.pyc'>

# Exemple de mean-field multi-filtre

In [ ]:
mean_field = mf.MeanField(5,5,2,2,1)

filter1 = np.zeros((5,5,2,2)) # Valeur identique à celle de ses voisins
for x,y in [(-1,0),(1,0),(0,-1),(0,1)]:
    for a in range(2):
        for b in range(2):
            filter1[x,y,a,b] = 1 if a != b else -1
filter2 = -filter1 # Filtre opposé.

weights = np.zeros((50,2,5,5,2,2),dtype=np.float32)
weights[:,0] = filter1
weights[:,1] = filter2

unary = np.zeros((50,5,5,2),dtype=np.float32)
L1 = np.zeros((10,1),dtype=np.float32)
L1_b = np.zeros((1),dtype=np.float32)-1.5
L2 = np.zeros((1,2),dtype=np.float32)
L2_b = np.zeros((2),dtype=np.float32)
L1[1,0] =  1
L1[6,0] =  1

L2[0,0] = -20
L2[0,1] =  20

FNN = (L1, L1_b, L2, L2_b)
theta_mf, energy, theta_clip = mean_field.build_model(5*weights,unary,50,0.05,FNN)

(array([[ 0.],
       [ 1.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 1.],
       [ 0.],
       [ 0.],
       [ 0.]], dtype=float32), array([-1.5], dtype=float32), array([[-20.,  20.]], dtype=float32), array([ 0.,  0.], dtype=float32))
('E SHAPE', TensorShape([Dimension(2), Dimension(None), Dimension(5), Dimension(5), Dimension(2)]))
('UNARY SHAPE', (50, 5, 5, 2))
('Q shape', TensorShape([Dimension(None), Dimension(5), Dimension(5), Dimension(2)]))


In [ ]:
sess = tf.Session()
res,hl,fs = sess.run([tf.nn.softmax(theta_mf), mean_field._hidden_layer, mean_field._filter_selection], feed_dict={theta_clip: np.expand_dims(mean_field.theta_clip_nothing(),0)})

In [ ]:
print(res[0,:,:,0])

In [200]:
print(fs[:,:,0])

[[ 1.  1.  1.  1.  1.]
 [ 1.  0.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.]]


In [202]:
print(np.tensordot(hl,L2,1))

[[[ 18.1  -18.1 ]
  [  9.24  -9.24]
  [ 18.1  -18.1 ]
  [ 18.1  -18.1 ]
  [ 18.1  -18.1 ]]

 [[  9.24  -9.24]
  [ -9.24   9.24]
  [  9.24  -9.24]
  [  9.24  -9.24]
  [  9.24  -9.24]]

 [[ 18.1  -18.1 ]
  [  9.24  -9.24]
  [ 18.1  -18.1 ]
  [ 18.1  -18.1 ]
  [ 18.1  -18.1 ]]

 [[ 18.1  -18.1 ]
  [  9.24  -9.24]
  [ 18.1  -18.1 ]
  [ 18.1  -18.1 ]
  [ 18.1  -18.1 ]]

 [[ 18.1  -18.1 ]
  [  9.24  -9.24]
  [ 18.1  -18.1 ]
  [ 18.1  -18.1 ]
  [ 18.1  -18.1 ]]]
